In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset

In [20]:
dataset = load_dataset("json", data_files={"train": "nerData-train.json", "test": "nerData-test.json"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/studio-lab-user/.cache/huggingface/datasets/json/default-86370b8076d6d38d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
dataset["train"][0]

{'ner_entities': ['O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-PERSON',
  'B-VP',
  'O',
  'B-DATE',
  'I-DATE',
  'O',
  'B-PRODUCT',
  'I-PRODUCT',
  'B-PRODUCT',
  'O',
  'O',
  'B-ORG',
  'B-EVENT',
  'B-DATE',
  'O'],
 'sentence': "The CEO of Apple, Tim Cook, announced the release date of the new iPhone 13 at the company's event yesterday.",
 'tokens': ['The',
  'CEO',
  'of',
  'Apple',
  ',',
  'Tim Cook',
  'announced',
  'the',
  'release',
  'date',
  'of',
  'the',
  'new',
  'iPhone 13',
  'at',
  'the',
  "company's",
  'event',
  'yesterday',
  '.'],
 'ner_tags': [0,
  7,
  0,
  7,
  0,
  1,
  43,
  0,
  15,
  16,
  0,
  13,
  14,
  13,
  0,
  0,
  7,
  29,
  15,
  0],
 'id': '1-1'}

In [15]:
len(dataset["test"])

30

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [25]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [27]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [28]:
tokenized_set = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [29]:
tokenized_set.keys()

dict_keys(['train', 'test'])

In [30]:
tokenized_set["train"][0]

{'ner_entities': ['O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-PERSON',
  'B-VP',
  'O',
  'B-DATE',
  'I-DATE',
  'O',
  'B-PRODUCT',
  'I-PRODUCT',
  'B-PRODUCT',
  'O',
  'O',
  'B-ORG',
  'B-EVENT',
  'B-DATE',
  'O'],
 'sentence': "The CEO of Apple, Tim Cook, announced the release date of the new iPhone 13 at the company's event yesterday.",
 'tokens': ['The',
  'CEO',
  'of',
  'Apple',
  ',',
  'Tim Cook',
  'announced',
  'the',
  'release',
  'date',
  'of',
  'the',
  'new',
  'iPhone 13',
  'at',
  'the',
  "company's",
  'event',
  'yesterday',
  '.'],
 'ner_tags': [0,
  7,
  0,
  7,
  0,
  1,
  43,
  0,
  15,
  16,
  0,
  13,
  14,
  13,
  0,
  0,
  7,
  29,
  15,
  0],
 'id': '1-1',
 'input_ids': [101,
  1996,
  5766,
  1997,
  6207,
  1010,
  5199,
  5660,
  2623,
  1996,
  2713,
  3058,
  1997,
  1996,
  2047,
  18059,
  2410,
  2012,
  1996,
  2194,
  1005,
  1055,
  2724,
  7483,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

2023-06-05 12:52:38.091900: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-05 12:52:38.648990: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-05 12:52:38.651141: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 12:52:41.918072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [32]:
import evaluate

seqeval = evaluate.load("seqeval")

In [34]:
import numpy as np


label_list = [
  'O', 'B-PERSON', 'I-PERSON', 'B-NORP',
  'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG',
  'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC',
  'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE',
  'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT',
  'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY',
  'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL',
  'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART',
  'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE',
  'I-LANGUAGE', 'B-VERB', 'B-ORGANISATION', 'B-LOCATION',
  'I-ORGANISATION', 'B-PEOPLE', 'I-LOCATION', 'B-VP',
  'B-ADJECTIVE', 'B-ORGANIZATION', 'B-TEAM', 'B-CAUSE',
  'B-BUSINESS', 'I-BUSINESS', 'B-PARTNER', 'B-JOB_TITLE',
  'B-NEW_TYPE', 'I-NEW_TYPE', 'B-NOUN', 'B-RELEASE',
  'B-ARTIFACT', 'I-ARTIFACT', 'B-EVENTS', 'B-FRIENDS',
  'B-LIFE_EVENT', 'B-SEASON', 'B-PARTNERSHIP', 'B-ACTIVITY',
  'B-MEETING', 'B-POSITION', 'B-SPORTS', 'I-ORGANIZATION',
  'B-NEW_PROJECT', 'I-NEW_PROJECT', 'B-SENTIMENT', 'B-NEW'
]

#labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
id2label = {
    0:'O',
    1:'B-PERSON',
    2:'I-PERSON',
    3:'B-NORP',
    4:'I-NORP',
    5:'B-FAC',
    6:'I-FAC',
    7:'B-ORG',
  8:'I-ORG',
    9:'B-GPE',
    10:'I-GPE', 
    11:'B-LOC',
  12:'I-LOC',
    13:'B-PRODUCT',
    14:'I-PRODUCT',
    15:'B-DATE',
  16:'I-DATE',
    17:'B-TIME', 
    18:'I-TIME',
    19:'B-PERCENT',
  20:'I-PERCENT',
    21:'B-MONEY',
    22:'I-MONEY',
    23:'B-QUANTITY',
  24:'I-QUANTITY',
    25:'B-ORDINAL',
    26:'I-ORDINAL',
    27:'B-CARDINAL',
 28: 'I-CARDINAL',
    29:'B-EVENT', 
    30:'I-EVENT',
    31:'B-WORK_OF_ART',
  32:'I-WORK_OF_ART', 
    33:'B-LAW',
    34:'I-LAW',
    35:'B-LANGUAGE',
  36:'I-LANGUAGE',
    37:'B-VERB',
    38:'B-ORGANISATION',
    39:'B-LOCATION',
  40:'I-ORGANISATION',
    41:'B-PEOPLE',
    42:'I-LOCATION',
    43:'B-VP',
  44:'B-ADJECTIVE',
  45: 'B-ORGANIZATION', 
    46: 'B-TEAM',
    47:'B-CAUSE',
  48:'B-BUSINESS',
    49:'I-BUSINESS',
    50:'B-PARTNER',
    51:'B-JOB_TITLE',
  52:'B-NEW_TYPE',
    53:'I-NEW_TYPE',
    54:'B-NOUN', 
    55:'B-RELEASE',
  56:'B-ARTIFACT',
    57:'I-ARTIFACT',
    58:'B-EVENTS',
    59:'B-FRIENDS',
  60:'B-LIFE_EVENT',
    61:'B-SEASON',
    62:'B-PARTNERSHIP',
    63:'B-ACTIVITY',
  64:'B-MEETING',
    65:'B-POSITION', 
    66:'B-SPORTS', 
    67:'I-ORGANIZATION',
  68:'B-NEW_PROJECT',
    69:'I-NEW_PROJECT', 
    70:'B-SENTIMENT', 
    71:'B-NEW'}

In [38]:
label2id = {
      'O':0,
    'B-PERSON':1,
    'I-PERSON':2,
    'B-NORP':3,
    'I-NORP':4,
    'B-FAC':5,
    'I-FAC':6,
    'B-ORG':7,
  'I-ORG':8,
    'B-GPE':9,
    'I-GPE':10, 
    'B-LOC':11,
  'I-LOC':12,
    'B-PRODUCT':13,
    'I-PRODUCT':14,
    'B-DATE':15,
  'I-DATE':16,
    'B-TIME':17, 
    'I-TIME':18,
    'B-PERCENT':19,
  'I-PERCENT':20,
    'B-MONEY':21,
    'I-MONEY':22,
    'B-QUANTITY':23,
  'I-QUANTITY':24,
    'B-ORDINAL':25,
    'I-ORDINAL':26,
    'B-CARDINAL':27,
 'I-CARDINAL':28,
    'B-EVENT':29, 
    'I-EVENT':30,
    'B-WORK_OF_ART':31,
  'I-WORK_OF_ART':32, 
    'B-LAW':33,
    'I-LAW':34,
    'B-LANGUAGE':35,
  'I-LANGUAGE':36,
    'B-VERB':37,
    'B-ORGANISATION':38,
    'B-LOCATION':39,
  'I-ORGANISATION':40,
    'B-PEOPLE':41,
    'I-LOCATION':42,
    'B-VP':43,
  'B-ADJECTIVE':44,
'B-ORGANIZATION':45, 
    'B-TEAM':46,
    'B-CAUSE':47,
  'B-BUSINESS':48,
    'I-BUSINESS':49,
    'B-PARTNER':50,
    'B-JOB_TITLE':51,
  'B-NEW_TYPE':52,
   'I-NEW_TYPE':53,
   'B-NOUN':54, 
    'B-RELEASE':55,
  'B-ARTIFACT':56,
   'I-ARTIFACT':57,
   'B-EVENTS':58,
   'B-FRIENDS':59,
  'B-LIFE_EVENT':60,
  'B-SEASON':61,
  'B-PARTNERSHIP':62,
  'B-ACTIVITY':63,
  'B-MEETING':64,
  'B-POSITION':65, 
  'B-SPORTS':66, 
  'I-ORGANIZATION':67,
  'B-NEW_PROJECT':68,
  'I-NEW_PROJECT':69, 
  'B-SENTIMENT':70, 
  'B-NEW':71}

In [39]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
#len(label2id.keys())
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(label2id.keys()), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [45]:
training_args = TrainingArguments(
    output_dir="my_test_ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_set["train"],
    eval_dataset=tokenized_set["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/tro9999/my_test_ner_model into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,3.980366,0.083333,0.076923,0.080000,0.240000
2,No log,3.916611,0.125000,0.076923,0.095238,0.300000


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 365703040 vs 365702928

In [46]:
text = "The CEO of Microsoft, Satya Nadella, announced yesterday that the company will be releasing a new product next month."

In [47]:
from transformers import pipeline

classifier = pipeline("ner", model="./my_test_ner_model")
classifier(text)

[{'entity': 'B-TIME',
  'score': 0.023089224,
  'index': 1,
  'word': 'the',
  'start': 0,
  'end': 3},
 {'entity': 'I-NORP',
  'score': 0.02550736,
  'index': 4,
  'word': 'microsoft',
  'start': 11,
  'end': 20},
 {'entity': 'I-LOC',
  'score': 0.023351219,
  'index': 5,
  'word': ',',
  'start': 20,
  'end': 21},
 {'entity': 'I-PERCENT',
  'score': 0.02314553,
  'index': 6,
  'word': 'sat',
  'start': 22,
  'end': 25},
 {'entity': 'I-NEW_PROJECT',
  'score': 0.020559443,
  'index': 7,
  'word': '##ya',
  'start': 25,
  'end': 27},
 {'entity': 'I-PERCENT',
  'score': 0.020316014,
  'index': 8,
  'word': 'nad',
  'start': 28,
  'end': 31},
 {'entity': 'I-NEW_PROJECT',
  'score': 0.02015071,
  'index': 9,
  'word': '##ella',
  'start': 31,
  'end': 35},
 {'entity': 'I-DATE',
  'score': 0.019782819,
  'index': 12,
  'word': 'yesterday',
  'start': 47,
  'end': 56},
 {'entity': 'I-NORP',
  'score': 0.020911902,
  'index': 15,
  'word': 'company',
  'start': 66,
  'end': 73},
 {'entity': 